<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b10e614c1f4c4a0935421b542b0dbd496a7bc99c1f9cb8865edbb307e46185be
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-10 10:17:45
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.42 C
-------------------
Today PnL: 1.12 L (0.8%)
Current PnL: -17.69 L (-11.9%)
CY Booked + Current PnL: -6.66 L (-4.48%)
-------------------
Total profit:  1.98 L
Total loss:  -19.67 L
-------------------
Total Booked + Current PnL: 20.32 L (16.7%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.34 L (54.64%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.53,60.0,H-LC,11.16,172865.0,16688.0,9404.0,0.41,10.69,5.44,16.71,22.0,1.77,1.32,29.99,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,56.0,M-LC,4.27,216664.0,15184.0,12025.0,1.08,7.54,5.55,13.51,60.0,1.26,1.66,21.08,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,104.70,60.0,M-SC,5.58,88163.0,-12614.0,12704.0,-0.84,-12.52,14.41,0.09,245.0,-0.99,0.68,15.83,OX40N,NTT,DURABLES
41,ITC,452.00,-37.92,50.0,H-LC,3.30,200221.0,83.0,20803.0,-0.15,0.04,10.39,10.44,4.0,0.00,1.53,5.61,X40,NTT,FMCG
49,NATIONALUM,247.44,-45.32,65.0,H-MC,1.70,109509.0,9005.0,21639.0,-1.87,8.96,19.76,30.49,79.0,0.42,0.84,44.27,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,42.74,63.0,M-MC,7.08,225530.0,568.0,167095.0,1.17,0.25,74.09,74.53,192.0,0.00,1.73,32.59,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-20.65,43.0,H-MC,1.99,136448.0,660.0,42094.0,0.94,0.49,30.85,31.48,107.0,0.02,1.05,12.66,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.34,63.0,H-MC,1.74,248742.0,1011.0,74050.0,0.47,0.41,29.77,30.30,92.0,0.01,1.91,13.75,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.11,45.0,H-MC,3.49,137433.0,1300.0,29823.0,1.22,0.95,21.70,22.86,91.0,0.04,1.05,13.69,X40,ATH,INSURANCE
51,PGHH,17973.08,-31.08,50.0,H-MC,4.70,199485.0,-1335.0,70119.0,-0.67,-0.66,35.15,34.25,80.0,-0.02,1.53,4.50,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.07,45.0,H-LC,13.77,218715.0,1309.0,52185.0,1.46,0.60,23.86,24.61,48.0,0.03,1.68,55.75,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-13.94,47.0,H-LC,8.24,173039.0,-8720.0,43969.0,0.50,-4.80,25.41,19.39,86.0,-0.20,1.33,2.96,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,47.0,H-LC,10.17,229471.0,-12453.0,75725.0,2.42,-5.15,33.00,26.15,8.0,-0.16,1.76,8.44,X40,ATH,IT
56,RELIANCE,1533.00,-13.66,48.0,H-LC,6.50,215795.0,5429.0,23349.0,0.52,2.58,10.82,13.68,37.0,0.23,1.65,19.53,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-2.89,61.0,H-LC,1.31,240998.0,-5153.0,84373.0,3.28,-2.09,35.01,32.18,16.0,-0.06,1.85,33.19,X200,ATH,IT
50,NESTLEIND,1377.00,-8.48,59.0,H-LC,2.30,281736.0,16310.0,40485.0,0.15,6.14,14.37,21.40,11.0,0.40,2.16,13.20,XY25,NTT,FMCG
41,ITC,452.00,-37.92,50.0,H-LC,3.30,200221.0,83.0,20803.0,-0.15,0.04,10.39,10.44,4.0,0.00,1.53,5.61,X40,NTT,FMCG
20,CIPLA,1795.00,-19.44,54.0,H-LC,5.96,214748.0,10248.0,31160.0,1.02,5.01,14.51,20.25,10.0,0.33,1.65,14.18,X40N,BTT,PHARMA
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-18.69,75.0,H-SC,8.38,186145.0,-53270.0,326219.0,5.53,-22.25,175.25,114.01,133.0,-0.16,1.43,70.72,AR,ATH,IT
76,TRIDENT,48.00,-6.66,60.0,M-SC,5.97,72717.0,-19538.0,45841.0,4.73,-21.18,63.04,28.51,224.0,-0.43,0.56,24.43,XR,NTT,TEXTILES
16,BSOFT,836.99,-12.08,56.0,H-SC,16.39,67891.0,-42158.0,78584.0,3.77,-38.31,115.75,33.10,131.0,-0.54,0.52,12.81,XR,ATH,IT
72,TATAELXSI,9161.00,-19.17,58.0,H-MC,8.25,80584.0,-22068.0,47673.0,3.65,-21.50,59.16,24.94,98.0,-0.46,0.62,22.50,OX40N,NTT,IT
47,LTIM,7230.20,-2.89,61.0,H-LC,1.31,240998.0,-5153.0,84373.0,3.28,-2.09,35.01,32.18,16.0,-0.06,1.85,33.19,X200,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.00,-329.57,58.0,H-SC,8.69,97682.0,-35604.0,120198.0,-3.59,-26.71,123.05,63.47,139.0,-0.30,0.75,63.74,XR,NTT,CHEMICALS
46,LAOPALA,464.00,116.69,52.0,H-SC,3.07,68769.0,-32077.0,57903.0,-2.01,-31.81,84.20,25.61,142.0,-0.55,0.53,27.50,AR,NTT,CERAMICS
49,NATIONALUM,247.44,-45.32,65.0,H-MC,1.70,109509.0,9005.0,21639.0,-1.87,8.96,19.76,30.49,79.0,0.42,0.84,44.27,MH,ATH,METALS
8,ATULAUTO,844.00,3678.38,61.0,M-SC,7.61,117788.0,-29580.0,81392.0,-1.09,-20.07,69.10,35.16,236.0,-0.36,0.90,20.31,XY24,NTT,AUTO
77,TTKPRESTIG,770.00,104.70,60.0,M-SC,5.58,88163.0,-12614.0,12704.0,-0.84,-12.52,14.41,0.09,245.0,-0.99,0.68,15.83,OX40N,NTT,DURABLES


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,71.0,H-SC,12.83,130734.0,-11745.0,31272.0,2.00,-8.24,23.92,13.70,163.0,-0.38,1.00,54.30,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,104.70,60.0,M-SC,5.58,88163.0,-12614.0,12704.0,-0.84,-12.52,14.41,0.09,245.0,-0.99,0.68,15.83,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,64.0,H-SC,2.37,224739.0,-44928.0,81266.0,0.42,-16.66,36.16,13.47,138.0,-0.55,1.72,13.67,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.84,50.0,M-SC,3.02,140752.0,-33807.0,33837.0,0.66,-19.37,24.04,0.02,221.0,-1.00,1.08,22.43,OX40N,NTT,PAINTS
19,CERA,9475.0,-19.81,46.0,H-SC,2.28,114264.0,-29539.0,56286.0,0.06,-20.54,49.26,18.60,149.0,-0.52,0.88,26.09,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.76,52.0,H-LC,13.94,292349.0,-53607.0,177456.0,1.99,-15.50,60.70,35.80,1.0,-0.30,2.24,3.57,X40,BTT,IT
39,INFY,2275.00,-16.73,57.0,H-LC,10.13,324697.0,11809.0,159881.0,1.34,3.77,49.24,54.87,3.0,0.07,2.49,10.67,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,3.30,200221.0,83.0,20803.0,-0.15,0.04,10.39,10.44,4.0,0.00,1.53,5.61,X40,NTT,FMCG
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,49.0,H-LC,12.53,248304.0,-13315.0,132520.0,0.80,-5.09,53.37,45.57,7.0,-0.10,1.90,5.66,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,42.74,63.0,M-MC,7.08,225530.0,568.0,167095.0,1.17,0.25,74.09,74.53,192.0,0.00,1.73,32.59,XY24,BTT,STEEL
4,ANGELONE,3033.00,9.18,42.0,H-SC,6.36,193184.0,2178.0,55521.0,0.85,1.14,28.74,30.21,157.0,0.04,1.48,22.26,X40N,NTT,FINANCE
15,BLUESTARCO,2326.38,9.17,53.0,H-MC,3.53,188480.0,23810.0,44161.0,-0.12,14.46,23.43,41.28,89.0,0.54,1.44,23.60,X40N,ATH,AC
84,VOLTAS,1918.49,0.60,58.0,H-MC,3.30,211050.0,19308.0,76717.0,-0.38,10.07,36.35,50.08,99.0,0.25,1.62,17.55,XY25,ATH,AC
30,HAVELLS,2069.16,0.34,63.0,H-MC,1.74,248742.0,1011.0,74050.0,0.47,0.41,29.77,30.30,92.0,0.01,1.91,13.75,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-45.32,65.0,H-MC,1.70,109509.0,9005.0,21639.0,-1.87,8.96,19.76,30.49,79.0,0.42,0.84,44.27,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.53,60.0,M-SC,2.50,102128.0,10630.0,68119.0,-0.69,11.62,66.70,86.07,223.0,0.16,0.78,48.31,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.91,51.0,H-SC,12.17,137185.0,13849.0,119913.0,0.53,11.23,87.41,108.45,119.0,0.12,1.05,36.09,AR,ATH,MISC
86,WIPRO,420.00,-9.71,59.0,M-LC,6.49,158354.0,7409.0,102043.0,2.51,4.91,64.44,72.51,53.0,0.07,1.21,11.23,XR,NTT,IT
12,BANKINDIA,190.00,-31.68,56.0,H-MC,5.48,176947.0,-2861.0,115281.0,2.02,-1.59,65.15,62.52,88.0,-0.02,1.36,31.95,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6755.56,54.0,L-SC,6.09,75225.0,-18585.0,95205.0,0.68,-19.81,126.56,81.67,269.0,-0.20,0.58,45.68,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-4.34,46.0,H-SC,14.69,88976.0,-11987.0,106522.0,1.95,-11.87,119.72,93.63,148.0,-0.11,0.68,32.53,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,60.0,L-SC,42.26,91200.0,-22349.0,62399.0,1.73,-19.68,68.42,35.27,268.0,-0.36,0.70,123.21,XR,NTT,HOTELS
48,MASFIN,398.61,-15.99,59.0,H-SC,13.02,95400.0,-2580.0,24184.0,-0.34,-2.63,25.35,22.05,152.0,-0.11,0.73,38.67,XR,ATH,FINANCE
82,VALIANTORG,838.00,-329.57,58.0,H-SC,8.69,97682.0,-35604.0,120198.0,-3.59,-26.71,123.05,63.47,139.0,-0.30,0.75,63.74,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,60.0,L-SC,42.26,91200.0,-22349.0,62399.0,1.73,-19.68,68.42,35.27,268.0,-0.36,0.70,123.21,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-171.59,65.0,M-SC,3.89,82992.0,-19218.0,112612.0,1.47,-18.80,135.69,91.37,208.0,-0.17,0.64,34.72,XY25,NTT,FINANCE
8,ATULAUTO,844.00,3678.38,61.0,M-SC,7.61,117788.0,-29580.0,81392.0,-1.09,-20.07,69.10,35.16,236.0,-0.36,0.90,20.31,XY24,NTT,AUTO
75,TITAGARH,1548.00,-3.20,64.0,H-SC,7.11,158742.0,-33273.0,112151.0,2.62,-17.33,70.65,41.08,158.0,-0.30,1.22,34.51,XY24,NTT,ENGINEERING
71,TANLA,1963.11,-18.69,75.0,H-SC,8.38,186145.0,-53270.0,326219.0,5.53,-22.25,175.25,114.01,133.0,-0.16,1.43,70.72,AR,ATH,IT


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.36
1,25,44.48
2,50,75.13


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.96
LC    35.39
MC    23.73
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.69
X40      16.06
X40N     13.12
XY25     10.28
XR        9.54
AR        8.40
OX40N     7.79
X200      1.85
MH        1.78
X5K       1.45
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.83
H-SC    25.12
H-MC    20.63
M-SC    14.15
M-LC     7.45
M-MC     2.77
L-SC     1.69
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.03,-14.76,72.86
FMCG,12.26,-3.72,38.88
FINANCE,9.11,-17.64,59.94
BANKS,7.74,-12.92,60.86
PAINTS,6.17,-14.79,40.48
MISC,6.02,-16.89,83.89
ELECTRICAL,5.88,-7.49,66.73
AC,3.81,2.57,30.43
AUTO,3.49,-15.03,66.90


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2722668.0,22
AR,1176410.0,9
XR,1049685.0,13
X40,872289.0,10
X40N,563297.0,10
OX40N,444769.0,10
XY25,411397.0,7
SR,196694.0,2
X5K,128546.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2749569.0,28
M-SC,1400334.0,17
H-LC,1365997.0,15
H-MC,1153942.0,15
M-LC,405529.0,5
M-MC,317017.0,2
L-SC,241972.0,3
L-MC,59667.0,1
L-LC,39648.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,910677.0,6
M-SC,XY24,818746.0,7
H-SC,AR,645525.0,5
H-LC,X40,585142.0,5
H-SC,XR,501130.0,6
H-MC,XY24,461128.0,4
H-LC,AR,371114.0,2
M-MC,XY24,317017.0,2
H-SC,X40N,245091.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
